# Carga Train y Test

In [12]:
import pandas as pd
# Cargar datos de entrenamiento y prueba
train_data = pd.read_excel("../phishing_train.xlsX")
test_data = pd.read_excel("../phishing_test.xlsx")

# Random Forest

In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Separar características (X) y etiquetas (y) en el conjunto de entrenamiento
X_train = train_data.drop(columns=['label'])  # Características de entrenamiento
y_train = train_data['label']  # Etiquetas de entrenamiento

# Convertir todas las columnas categóricas a cadenas (string)
X_train = X_train.astype({col: str for col in X_train.select_dtypes(include=['object', 'category']).columns})
test_data = test_data.astype({col: str for col in test_data.select_dtypes(include=['object', 'category']).columns})

# Identificar columnas categóricas
categorical_columns = X_train.select_dtypes(include=['object']).columns

# Transformar columnas categóricas con One-Hot Encoding
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ],
    remainder='passthrough'  # Mantener las columnas numéricas sin cambios
)

# Aplicar la transformación al conjunto de entrenamiento y prueba
X_train_transformed = column_transformer.fit_transform(X_train)
X_test_transformed = column_transformer.transform(test_data)

# Entrenar el modelo de Random Forest
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train_transformed, y_train)

# Predecir probabilidades para los datos de prueba
y_proba_test = model.predict_proba(X_test_transformed)[:, 1]

# Crear un DataFrame con las columnas 'Id' y 'Probability'
submission = pd.DataFrame({
    'Id': test_data['Id'],  # Asegurarse de que la columna 'Id' está presente en test_data
    'Probability': y_proba_test
})

# Guardar el archivo de envío en formato CSV
submission.to_csv("RFsubmission.csv", index=False)

print("Archivo de envío generado: 'submission.csv'")


Archivo de envío generado: 'submission.csv'


# Gradient Boosting